# Loading a local model

This script shows how to load a local model .h5 using our modules.

## Set configuration

There is a file named as cofigure.json in the /doc path. The methods implemented in our modules uses this file to load configures values that will be used, values as model type, nodes, epochs to train, data test size and others. Things like fitted models path and a remote model path are configure there too.

In [1]:
# This is a configure.json file example.
import json

# Load the local configure.json and print it
configure_json = open('../doc/configure.json', 'r')
print(json.dumps(json.load(configure_json), indent=4))

{
    "ncovid": "ML COVID-19 configure file",
    "author": "NatalNet NCovid",
    "published_at": 2021,
    "folder_configs": {
        "docs_path": "../doc/",
        "data_path": "../dbs/",
        "model_path": "fitted_model/",
        "model_path_remote": "https://",
        "glossary_file": "glossary.json"
    },
    "model_configs": {
        "type_used": "Artificial",
        "is_predicting": "False",
        "Artificial": {
            "model": "lstm",
            "nodes": 300,
            "epochs": 100,
            "dropout": 0.1,
            "batch_size": 64,
            "earlystop": 30,
            "is_output_in_input": "True",
            "data_configs": {
                "is_accumulated_values": "False",
                "is_apply_moving_average": "True",
                "window_size": 7,
                "data_test_size_in_days": 35,
                "type_norm": ""
            },
            "Autoregressive": {
                "model": "arima",
                "p": 1,
    

To load this set of configurations, import the configs_manner.py file.

In [2]:
# If this script is running in another folder, change the base path to the /src folder.
import sys
sys.path.append("../src")

import configs_manner

# Priting some configures variabels.
print("Model infos: \n", configs_manner.model_infos)
print("\n")
print("Models path: \n", configs_manner.model_path)

Model infos: 
 {'model_nodes': 300, 'model_epochs': 100, 'model_dropout': 0.1, 'model_batch_size': 64, 'model_earlystop': 30, 'model_is_output_in_input': True, 'data_is_accumulated_values': False, 'data_is_apply_moving_average': True, 'data_window_size': 7, 'data_test_size_in_days': 35, 'data_type_norm': ''}


Models path: 
 ../dbs/fitted_model/


To configure any path model param, just change the value in the configure.json file.

## Create model

To create a model constructor, is needed to collect and create a dataset construct before, because the modelus has to know some information to create the model archtecture automatically (deppending of the number of data features). 

In this script it'll be do a remote data request, so before to create a data constructor, it is necessary declare the remote repository, the locality, the features to get, also the start and finish date. So, we need to create a data constructor and use a collect method. For more details see [Loading remote data](loading_remote_data.ipynb) file.

In [4]:
# specif code to the remote repository data.
repo = "p971074907"
# coutry and state acronym splited by a ":"
path = "brl:rn"
# columns (or features) to be extracted from the database, each one splited by a ":"
feature = "date:newDeaths:newCases:"
# start date for the data request.
begin = "2020-05-01"
# finish date for the data request.
end = "2021-07-01"

# import the data_manner.py file. (taking into account that you are in src/ path)
import data_manner

# creating the DataConstructor instance
data_constructor = data_manner.DataConstructor()
# collect data from the remote repository.
collected_data = data_constructor.collect_dataframe(path, repo, feature, begin, end)

# building the data test to set up the number of features to model model archtecture.
test = data_constructor.build_test(collected_data)

To create a model instance, it's needed to import the desired model manner file and call the creating() method to mount the model architecture. In this example it'll be used a LSTM model. See [Creating a model](creating_a_model.ipynb) for more details.

In [5]:
# import the lstm_manner.py file. (taking into account that you are in src/ path)
from models.artificial import lstm_manner

# creating  a model instance
lstm_model = lstm_manner.ModelLSTM(path)
# set up the model architecture
lstm_model.creating()
print(lstm_model)


LSTM Model
	input, output and timesteps: 7
	lstm nodes: 300
	features: 2
	dropout: 0.1


## Load local model

To load a desired model o have to know it uuid. To see it, open the metadata.json created when you saved ([Training and saving model](training_and_saving_a_model.ipynb)) the model and find "model_id". Also, you can see the .h5 file name in the fitted models path.

In [6]:
lstm_model.loading("cee94ec2-ac6e-11ec-84ad-48a47252b4f8")

This method laods the trained weights from saved model and makes the new instance equal as the trained.